In [2]:
import pandas as pd
import pandas.io.sql as psql
import os
import sys
from datetime import date
current_date = date.today()
#%load_ext autotime
import re
import csv
import pandas as pd
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
#from matplotlib_venn import venn2, venn3
import warnings
import time
import datetime as dt
import itertools
from more_itertools import unique_everseen
warnings.filterwarnings("ignore")
#import pandasql as ps
import seaborn as sns
import json
from ast import literal_eval
import ast
from fuzzywuzzy import process
from nltk.corpus import words

In [ ]:
total = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\total_summary_stats_updated.csv")

In [ ]:
patient_id = total["patient_id"].to_list()
patient_id = patient_id[:10]

In [ ]:
matches = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\apoe_1_matches.csv")

In [ ]:
# ONLY RUN ONCE #

# vertifying that all match records can be found
for i in range(len(patient_id)):
    specific_matches = matches[matches["PatientID"] == patient_id[i]]
    if (len(specific_matches) <= 0):
        print("NOT FOUND: ", patient_id[i])
    #specific_matches.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\Specific_Matches\patient_{}.csv".format(i+1))

In [ ]:
p1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\Specific_Matches\patient_1.csv")

In [ ]:
unique_ids = matches["PatientID"].unique().tolist()
print(len(unique_ids))

In [ ]:
def get_match(query, choices, limit = 1):
    results = process.extract(query, choices, limit = limit)
    return results

In [ ]:
# interesting match cases
# patient_2 in specific study row 6, cream that has mci inside it

In [ ]:
subset_loc = p1[p1["PatientID"] == 'Z6352398']['regex_location'].to_list()
subset_loc_clone = p1[p1["PatientID"] == 'Z6352398']['regex_location'].to_list()
#print(subset_loc)
encounters = p1[p1["PatientID"] == 'Z6352398']['PatientEncounterID'].unique().tolist()

#print(len(subset_loc))
#print(subset_loc)
#print(subset_loc[0])
for i in range(len(subset_loc)):
    subset_loc[i] = subset_loc[i][1:len(subset_loc[i]) - 1]
    subset_loc[i] = eval(subset_loc[i])
    #subset_loc[i] = tuple(map(int, subset_loc[i].split(", ")))
    subset_loc_clone[i] = subset_loc_clone[i][1:len(subset_loc_clone[i]) - 1]
    subset_loc_clone[i] = eval(subset_loc_clone[i])
    
subset_loc = pd.Series(subset_loc)
subset_loc = subset_loc.unique().tolist()

subset_loc_clone = pd.Series(subset_loc_clone)
subset_loc_clone = subset_loc_clone.unique().tolist()

print("CLONE: ", subset_loc_clone)
print(subset_loc)
print("\n", subset_loc[0][0])
print(len(subset_loc))
if (len(subset_loc) != len(encounters)):
    print("Houston, we got a problem", i, len(subset_loc), len(encounters))

total_merged = []  

for i in range(len(subset_loc)):
    merged = []

    start = 0
    end = 0
    length = 0
    
    if (len(subset_loc[i]) == 2):
        start = subset_loc[i][0]
        end = subset_loc[i][1]
        length = len(p1[(p1["regex_location"] == "[" +str(subset_loc[i]) + "]") & (p1["PatientEncounterID"] == int(encounters[i]))]['NoteTXT'].values[0])
        if (start < 100):
            start = 100
        elif (end > length - 100):
             end = length - 100
        subset_loc[i] = [(start - 100, end + 100)]
        
    else:
        loc_list = []
        for j in range(len(subset_loc[i])):
            start = subset_loc[i][j][0]
            end = subset_loc[i][j][1]
            loc_list.append((start, end))
        length = len(p1[(p1["regex_location"] == str(loc_list)) & (p1["PatientEncounterID"] == int(encounters[i]))]['NoteTXT'].values[1])
        for k in range(len(loc_list)):
            start = loc_list[k][0]
            end = loc_list[k][1]
            if (start < 100):
                start = 100
            elif (end > length - 100):
                end = length - 100
            loc_list[k] = (start - 100, end + 100)
        subset_loc[i] = loc_list
    
    subset_loc[i].sort(key = lambda x : x[0])
    
    print("\nSUBSET_LOC", subset_loc[i])
    
    for interval in subset_loc[i]:
        if not merged:
            merged.append(interval)
            continue

        prevInterval = merged.pop()
        if prevInterval[0] <= interval[0] <= prevInterval[1]:
            startTime = prevInterval[0]
            endTime = prevInterval[1] if prevInterval[1] > interval[1] else interval[1]
            merged.append((startTime, endTime))
        else:
            merged.append(prevInterval)
            merged.append(interval)
    total_merged.append(merged)
    
total_merged
#     print("start", start)
#     print("end", end)
    #print("matches, ", p1[(p1["regex_location"] == "[" +str(subset_loc[j]) + "]") & (p1["PatientEncounterID"] == int(encounters[j]))]['NoteTXT'].values)
    #print("length, ", length) 
    

In [ ]:
print(subset_loc_clone)

In [ ]:
total_note_txt = []
print(len(total_merged))
for i in range(len(total_merged)):
    curr = ""
    if (len(total_merged[i]) == 1):
        note = p1[(p1["regex_location"] == "[" +str(subset_loc_clone[i]) + "]") & (p1["PatientEncounterID"] == int(encounters[i]))]['NoteTXT'].values[0]
        #print(note)
        #print(total_merged[i][0][0], total_merged[i][0][1])
        curr = note[total_merged[i][0][0]:total_merged[i][0][1]]
    else:
        for j in range(len())
    total_note_txt.append(curr)
    
total_note_txt

In [ ]:
df = pd.DataFrame(total_note_txt, columns = ["text"])
df = df.drop_duplicates(subset='text', keep="first")
df = df.replace(r'^\s*$', np.NaN, regex=True)
df.dropna()
total_note_txt = df["text"].to_list()

In [ ]:
total_note_txt

In [ ]:
p1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\3\apoe_3_matches.csv")

In [20]:
p1.head()

,Unnamed: 0,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT,regex_location,regex_sent,regex_match,match_count
0,0,100365175,10059358563,Z8714148,3.204238e+09,2018-12-12 00:00:00.0000000,Office Visit,2250389677,Progress Notes,2,2.215690e+09,1.0,----------------------------------------------...,"[(9625, 9634)]","""he drinks alcohol. He reports that he does no...","[""Alzheimer""]",1
1,1,100365175,10059358563,Z8714148,3.204238e+09,2018-12-12 00:00:00.0000000,Office Visit,2251218506,Letter,2,2.216533e+09,1.0,----------------------------------------------...,"[(10155, 10164)]","""he drinks alcohol. He reports that he does no...","[""Alzheimer""]",1
2,2,100365175,10059358563,Z8714148,3.320352e+09,2020-11-11 00:00:00.0000000,Telemedicine,4835633269,Progress Notes,2,4.825891e+09,1.0,----------------------------------------------...,"[(13840, 13849)]","""of Onset \u2022 Other Mother frequent falls \...","[""Alzheimer""]",1
3,3,100365175,10059358563,Z8714148,3.320352e+09,2020-11-11 00:00:00.0000000,Telemedicine,4840625130,Letter,2,4.830916e+09,1.0,----------------------------------------------...,"[(14369, 14378)]","""of Onset \u2022 Other Mother frequent falls \...","[""Alzheimer""]",1
4,4,100365365,5361447,Z8715609,3.052224e+09,2009-11-23 00:00:00.0000000,System Generated,336757101,Discharge Summary,2,3.224977e+08,1.0,----------------------------------------------...,"[(3785, 3794)]","""don\\'t think so.' Denies visual hallucinatio...","[""Cognition""]",1


In [21]:
total_merged = []

for index, row in p1.iterrows():
    curr = str(row["regex_location"])
    curr = curr[1:len(curr) - 1]
    curr = list((eval(curr)))
    #print("CURR: ", len(curr))
    # print(curr)
    merged = []

    start = 0
    end = 0
    length = 0
    
    if (len(curr) == 2 and isinstance(curr[1], int)):
        start = curr[0]
        end = curr[1]
        #print(start, end)
        length = len(row["NoteTXT"])
        if (start < 100):
            start = 100
        elif (end > length - 100):
            end = length - 100
        curr = [(start - 100, end + 100)]
        
    else:
        loc_list = []
        for j in range(len(curr)):
            start = curr[j][0]
            end = curr[j][1]
            loc_list.append((start, end))
        length = len(row["NoteTXT"])
        for k in range(len(loc_list)):
            start = loc_list[k][0]
            end = loc_list[k][1]
            if (start < 100):
                start = 100
            elif (end > length - 100):
                end = length - 100
            loc_list[k] = (start - 100, end + 100)
        curr = loc_list
        
    for interval in curr:
        if not merged:
            merged.append(interval)
            continue

        prevInterval = merged.pop()
        if prevInterval[0] <= interval[0] <= prevInterval[1]:
            startTime = prevInterval[0]
            endTime = prevInterval[1] if prevInterval[1] > interval[1] else interval[1]
            merged.append((startTime, endTime))
        else:
            merged.append(prevInterval)
            merged.append(interval)
            
    total_merged.append(merged)
p1["merged_row_location"] = total_merged

In [22]:
p1

,Unnamed: 0,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT,regex_location,regex_sent,regex_match,match_count,merged_row_location
0,0,100365175,10059358563,Z8714148,3.204238e+09,2018-12-12 00:00:00.0000000,Office Visit,2250389677,Progress Notes,2,2.215690e+09,1.0,----------------------------------------------...,"[(9625, 9634)]","""he drinks alcohol. He reports that he does no...","[""Alzheimer""]",1,"[(9525, 9734)]"
1,1,100365175,10059358563,Z8714148,3.204238e+09,2018-12-12 00:00:00.0000000,Office Visit,2251218506,Letter,2,2.216533e+09,1.0,----------------------------------------------...,"[(10155, 10164)]","""he drinks alcohol. He reports that he does no...","[""Alzheimer""]",1,"[(10055, 10264)]"
2,2,100365175,10059358563,Z8714148,3.320352e+09,2020-11-11 00:00:00.0000000,Telemedicine,4835633269,Progress Notes,2,4.825891e+09,1.0,----------------------------------------------...,"[(13840, 13849)]","""of Onset \u2022 Other Mother frequent falls \...","[""Alzheimer""]",1,"[(13740, 13949)]"
3,3,100365175,10059358563,Z8714148,3.320352e+09,2020-11-11 00:00:00.0000000,Telemedicine,4840625130,Letter,2,4.830916e+09,1.0,----------------------------------------------...,"[(14369, 14378)]","""of Onset \u2022 Other Mother frequent falls \...","[""Alzheimer""]",1,"[(14269, 14478)]"
4,4,100365365,5361447,Z8715609,3.052224e+09,2009-11-23 00:00:00.0000000,System Generated,336757101,Discharge Summary,2,3.224977e+08,1.0,----------------------------------------------...,"[(3785, 3794)]","""don\\'t think so.' Denies visual hallucinatio...","[""Cognition""]",1,"[(3685, 3894)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246740,246761,101641676,10070410567,Z11443906,3.310237e+09,2020-07-08 00:00:00.0000000,Telemedicine - audio only,4221710901,Letter,2,4.207052e+09,1.0,----------------------------------------------...,"[(13262, 13271)]","""(617-724-2076). Best regards, Jacob Dal-Bianc...","[""cognition""]",1,"[(13162, 13371)]"
246741,246762,101641676,10070410567,Z11443906,3.310237e+09,2020-07-08 00:00:00.0000000,Telemedicine - audio only,4221760690,Progress Notes,2,4.207102e+09,1.0,----------------------------------------------...,"[(12738, 12747)]","""(617-724-2076). Best regards, Jacob Dal-Bianc...","[""cognition""]",1,"[(12638, 12847)]"
246742,246763,101641676,10070410567,Z11443906,3.314701e+09,2021-02-03 00:00:00.0000000,Telemedicine,5249047067,Progress Notes,2,5.242458e+09,1.0,----------------------------------------------...,"[(12542, 12551)]","""(617-724-2076). Best regards, Jacob Dal-Bianc...","[""cognition""]",1,"[(12442, 12651)]"
246743,246764,101641676,10070410567,Z11443906,3.314701e+09,2021-02-03 00:00:00.0000000,Telemedicine,5250596691,Letter,2,5.244019e+09,1.0,----------------------------------------------...,"[(13070, 13079)]","""(617-724-2076). Best regards, Jacob Dal-Bianc...","[""cognition""]",1,"[(12970, 13179)]"


In [23]:
total_merged_note_txt = []
for index, row in p1.iterrows():
    curr = row["merged_row_location"]
    merged_note_txt = ""
    
    for i in range(len(curr)):
        start = curr[i][0]
        end = curr[i][1]
        
        merged_note_txt += " ------- " + row["NoteTXT"][start:end]
    total_merged_note_txt.append(merged_note_txt)
    
p1["merged_regex_sent"] = total_merged_note_txt

In [24]:
len(p1)

246745

In [25]:
p1['merged_regex_sent'] = p1['merged_regex_sent'].str.split(' ',1).str[1].str.rpartition(' ')[0]

In [26]:
#p1['NoteTXT'] = p1['NoteTXT'].astype(str)
p1 = p1.drop_duplicates(subset='merged_regex_sent', keep="first")
p1 = p1.replace(r'^\s*$', np.NaN, regex=True)
p1.reset_index(drop = True)

,Unnamed: 0,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT,regex_location,regex_sent,regex_match,match_count,merged_row_location,merged_regex_sent
0,0,100365175,10059358563,Z8714148,3.204238e+09,2018-12-12 00:00:00.0000000,Office Visit,2250389677,Progress Notes,2,2.215690e+09,1.0,----------------------------------------------...,"[(9625, 9634)]","""he drinks alcohol. He reports that he does no...","[""Alzheimer""]",1,"[(9525, 9734)]",------- at he drinks alcohol. He reports that ...
1,2,100365175,10059358563,Z8714148,3.320352e+09,2020-11-11 00:00:00.0000000,Telemedicine,4835633269,Progress Notes,2,4.825891e+09,1.0,----------------------------------------------...,"[(13840, 13849)]","""of Onset \u2022 Other Mother frequent falls \...","[""Alzheimer""]",1,"[(13740, 13949)]",------- ge of Onset • Other Mother frequent fa...
2,4,100365365,5361447,Z8715609,3.052224e+09,2009-11-23 00:00:00.0000000,System Generated,336757101,Discharge Summary,2,3.224977e+08,1.0,----------------------------------------------...,"[(3785, 3794)]","""don\\'t think so.' Denies visual hallucinatio...","[""Cognition""]",1,"[(3685, 3894)]","------- don't think so."" Denies visual halluc..."
3,5,100365365,5361447,Z8715609,3.076260e+09,2015-07-09 00:00:00.0000000,Office Visit,502337068,Progress Notes,2,4.822966e+08,1.0,----------------------------------------------...,"[(3719, 3727)]","""she reports is in question Family Issues/ Soc...","[""cerebral""]",1,"[(3619, 3827)]",------- osis she reports is in question Family...
4,6,100365365,5361447,Z8715609,3.076466e+09,2015-11-02 00:00:00.0000000,Office Visit,664173972,MR AVS Snapshot,2,6.346699e+08,1.0,----------------------------------------------...,"[(6304, 6319)]","""s/p Prediabetes Squamous cell carcinoma (HCC)...","[""Cerebrovascular""]",1,"[(6204, 6419)]",------- Depression–bipolar s/p Prediabetes Squ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74672,246755,101641676,10070410567,Z11443906,3.223658e+09,2019-10-31 00:00:00.0000000,Office Visit,3163903978,Progress Notes,2,3.150948e+09,1.0,----------------------------------------------...,"[(795, 804), (880, 886)]","""Life Planning Discussed: Yes Depression Scree...","[""Cognition"", ""memory""]",2,"[(695, 986)]",------- of Life Planning Discussed: Yes Depres...
74673,246757,101641676,10070410567,Z11443906,3.230882e+09,2018-12-27 00:00:00.0000000,Anesthesia Event,2276892684,Anesthesia Preprocedure Evaluation,2,2.242909e+09,1.0,----------------------------------------------...,"[(1022, 1030)]","""dose for Plavix 12/27/2018) NEURO/MUSCULAR/SK...","[""cerebral""]",1,"[(922, 1130)]",------- ast dose for Plavix 12/27/2018) NEURO/...
74674,246758,101641676,10070410567,Z11443906,3.278084e+09,2020-11-11 00:00:00.0000000,Office Visit,4841434446,Progress Notes,2,4.831730e+09,1.0,----------------------------------------------...,"[(1323, 1332), (1349, 1355)]","""opioid medication(s). Depression Screening: D...","[""Cognition"", ""memory""]",2,"[(1223, 1455)]",------- scription opioid medication(s). Depres...
74675,246760,101641676,10070410567,Z11443906,3.284372e+09,2019-12-23 00:00:00.0000000,Hospital Encounter,3419611263,Progress Notes,2,3.563963e+09,1.0,----------------------------------------------...,"[(16160, 16163)]","""placement. After negative aspiration a 3ml so...","[""mci""]",1,"[(16060, 16263)]",------- le placement. After negative aspiratio...


In [27]:
p1.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\2\apoe_2_matches_updated.csv", index = False)

In [ ]:
x = ((7509, 7518), (7735, 7741), (7745, 7754), (7858, 7862))
y = []
for i in range(len(x)):
    a = x[i][0]
    b = x[i][1]
    y.append((a,b))
print(str(y))
z = len(p1[(p1["regex_location"] == str(y)) & (p1["PatientEncounterID"] == int(encounters[21]))]['NoteTXT'].values[1])
z

In [ ]:
x = [(181, 390),
  (479, 688),
  (2048, 2269),
  (6184, 6390),
  (6584, 6821),
  (7450, 7659),
  (7716, 7932),
  (9864, 10072),
  (11192, 11648),
  (12118, 12627)]
y = []
for i in range(len(x)):
    a = x[i][0]
    b = x[i][1]
    y.append((a,b))
print(y)

In [ ]:
len(p1[(p1["regex_location"] == "[" +str(subset_loc[1]) + "]") & (p1["PatientEncounterID"] == int(encounters[1]))]['NoteTXT'].values[0])

In [ ]:
print(subset_loc_clone)